# Exercise Sheet 1 - The QR decomposition

In this exercise we will implement the QR-Algorithm using Householder transformations. Recall from the lecture:

For an arbitrary $v \neq 0$, we define $$P = I - 2 \frac{vv^\top}{v^\top v}.$$ These matrices are called Householder reflectors and are symmetric and orthogonal.

## (1) Householder Reflectors

We want to use Householder reflectors to zero out components in matrices. Let's start with a simple vector. Given an arbitrary vector $x \neq 0$, we want to find a Householder reflector $P$ as above such that $$ P x = k e_1,$$ where $k$ is a constant an $e_1$ the first unit vector.

**Task: Write a function that takes $x$ as an input and returns the matrix $P$ that projects $x$ to a multiple of the first unit vector.**

Hint: Consult your lecture notes if you don't remember how it works.

In [1]:
import numpy as np
import numpy.linalg as nplin

#An orthogonal matrix will not amplify the error term over iterations, as ∣∣QX∣∣=∣∣X∣∣.
#Multiplying this orthogonal matrix with vector 𝑋 will make all entries zero except the first one.
#Function will give you an orthogonal matrix for vector 𝑋.
def h(x):
    e1 = np.eye(len(x))[0]
    pro_x = nplin.norm(x) * e1
    dif = x - pro_x
    identity = np.identity(len(x))
    h = identity - (2 * np.outer(dif, dif)) / np.inner(dif,dif)
    return h


Apply your function to a random vector generated by
```
x = np.random.rand(5)
```
The result of the matrix vector multiplication may not be **exactly** what we wanted. Why is that?

In [2]:
#Generate five random numbers with a uniform distribution.
x = np.random.rand(5)

In [3]:
x #to see first five row of x

array([0.1902858 , 0.66031927, 0.03821414, 0.23641406, 0.90412092])

In [4]:
h(x)

array([[ 0.16395319,  0.56894131,  0.03292589,  0.20369801,  0.77900458],
       [ 0.56894131,  0.61282764, -0.02240652, -0.13861929, -0.53012329],
       [ 0.03292589, -0.02240652,  0.99870329, -0.0080222 , -0.03067941],
       [ 0.20369801, -0.13861929, -0.0080222 ,  0.95037015, -0.1898    ],
       [ 0.77900458, -0.53012329, -0.03067941, -0.1898    ,  0.27414573]])

In [5]:
np.dot(h(x),x).round(5)

array([ 1.16061, -0.     , -0.     , -0.     , -0.     ])

## (2) Application of Householder Reflectors

The function you wrote for the previous task is already the main tool for computing the QR decomposition. For a given matrix $A\in\mathbb{R}^{m,n}$, we want to compute the orthogonal matrix $Q$ such that $$A = Q R.$$ In the lecture, we discussed how to compute matrices $H_1, H_2, \dots, H_n$ with $Q^\top = H_4 H_3 \cdots  H_1$.

Let's start simple by just using our previous results.

**Task: Define the matrix**
```
M = np.array([[3,2,9],[0,5,6],[4,7,1]])
```
**and use the function from part (1) to project the entries from of first column to a multiple of the first unit vector. Print the matrix $M$ after the application of the projection.**

Remark: The task is not to treat the first column as a vector, but to determine the matrix $H_1$. This means that the other entries in the matrix also change when $H_1$ and $M$ are multiplied.

In [21]:
M = np.array([[3,2,9],
              [0,5,6],
              [4,7,1]])
H1=h(M.transpose()[0])
M1= np.matmul( H1,M)

In [7]:
H1

array([[ 0.6,  0. ,  0.8],
       [ 0. ,  1. ,  0. ],
       [ 0.8,  0. , -0.6]])

In [8]:
np.dot( H1,M.transpose()[0])

array([5., 0., 0.])

**Task: Create a zero below the diagonal in the second column, i.e., calculate the matrix $H_2$ and multiply it with your previous result.**

In [9]:
h2=h(M1.transpose()[1, 1:])
a=np.ones((1, 1))
b=np.zeros((1, len(h2)))
c=np.zeros((len(h2),1))
H2=np.bmat([[a, b],[c,h2]])
H2@M1

matrix([[ 5.00000000e+00,  6.80000000e+00,  6.20000000e+00],
        [ 0.00000000e+00,  5.63560112e+00,  2.27837275e+00],
        [ 0.00000000e+00,  1.27216467e-15, -8.62374731e+00]])

Now, that you have understood the concept, we can write a function that generalizes it to arbitrary $m$ by $n$ matrices.

**Task: Write a function that returns a list with the matrices $H_j \in\mathbb{R}^{m,m}$, $j = 1,2,\dots,n$. The input
is a $m\times n$ matrix $A$.**

Remark: Do not modify the input matrix A! Since $A$ can be rectangular, think about when to stop the algorithm.

In [10]:
def householder_qr(A):
    m, n = np.shape(A)
    matrices_H = []

    for j in range(min(m-1, n)):
        x = A[j:, j]
        e1 = np.eye(len(x))[0]
        v = np.sign(x[0]) * np.linalg.norm(x) * e1 + x
        v /= np.linalg.norm(v)  # Normalize v to make Hj orthogonal

        # Build Hj
        Hj = np.eye(m)
        Hj[j:, j:] -= 2 * np.outer(v, v)

        matrices_H.append(Hj)

    return matrices_H



Test your function using the matrix
```
A = np.array([[1,3,8,6,2],[2,5,6,3,4],[4,7,9,5,1], [2,4,6,1,7]])
```
by multiplying the matrices from your function with it!

In [11]:
A = np.array([[1,3,8,6,2],[2,5,6,3,4],[4,7,9,5,1], [2,4,6,1,7]])
H_matrices = householder_qr(A)

# Print the Householder matrices
for j, Hj in enumerate(H_matrices):
    print(f'H_{j+1}:\n{Hj}\n')

H_1:
[[-0.2        -0.4        -0.8        -0.4       ]
 [-0.4         0.86666667 -0.26666667 -0.13333333]
 [-0.8        -0.26666667  0.46666667 -0.26666667]
 [-0.4        -0.13333333 -0.26666667  0.86666667]]

H_2:
[[ 1.          0.          0.          0.        ]
 [ 0.         -0.52704628 -0.73786479 -0.42163702]
 [ 0.         -0.73786479  0.64346565 -0.20373391]
 [ 0.         -0.42163702 -0.20373391  0.88358062]]

H_3:
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.         -0.83205029 -0.5547002 ]
 [ 0.          0.         -0.5547002   0.83205029]]



In [12]:
Qt=nplin.multi_dot(H_matrices)

In [13]:
q=Qt.transpose()

In [14]:
r = np.matmul(Qt,A)

In [15]:
np.matmul(q,r)

array([[1., 3., 8., 6., 2.],
       [2., 5., 6., 3., 4.],
       [4., 7., 9., 5., 1.],
       [2., 4., 6., 1., 7.]])

## (3) QR Factorization

Now, we want a complete QR function.

 **Task: Write a function**

```
QR(A)
```

**that returns the orthogonal matrix $Q$ and the upper right triangular matrix $R$. The input is an $m\times n$ matrix $A$.**

In [16]:
def QR(A):
    Qt=nplin.multi_dot(H_matrices)
    q=Qt.transpose()
    r=np.matmul(Qt,A)
    return q,r

In [17]:
QR(A)

(array([[-0.2       , -0.4       , -0.8       , -0.4       ],
        [ 0.96976515, -0.20379123, -0.09135469, -0.09838197],
        [ 0.12702073,  0.890163  , -0.33770406, -0.27826524],
        [ 0.05849179,  0.0779678 , -0.48745286,  0.86769202]]),
 array([[  2.3645968 ,   5.37193802,   5.71272822,   2.40289089,
           4.01552387],
        [  2.90900512,   4.32405603,   4.05652639,   1.51740909,
          -0.17922732],
        [ -3.30843133,  -7.17051329, -12.91218182,  -7.25003722,
          -5.71529282],
        [  0.02555914,  -0.16899845,  -1.08852686,  -3.21878009,
           4.60205102]]))

The numpy library also provides a method to compute the QR factorization of a matrix. Read the documentation [here](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.qr.html) and compare the outputs $Q$ and $R$ of your function with the outputs from the numpy libray. Do you notice any differences? Can you explain the differences?

In [18]:
Q, R = nplin.qr(A)

In [19]:
Q,R

(array([[-0.2       , -0.60448772,  0.74413167, -0.20214514],
        [-0.4       , -0.62773725, -0.65405257, -0.13476342],
        [-0.8       ,  0.48824008,  0.0900791 , -0.33690856],
        [-0.4       , -0.04649906,  0.10182854,  0.90965311]]),
 array([[ -5.        ,  -9.8       , -13.6       ,  -6.8       ,
          -5.6       ],
        [  0.        ,  -1.72046505,  -4.48715886,  -3.11543672,
          -3.55717775],
        [  0.        ,   0.        ,   3.45042105,   3.05485632,
          -0.32506804],
        [  0.        ,   0.        ,   0.        ,  -2.39205078,
           5.08731926]]))

# (4) Additional Task 1

You can work the additional tasks in this section if you are already done with the main part of the problem sheet.

We first want to take a look at Hessenberg matrices, which often appear in numerical linear algebra, e.g., for the approximation of eigenvalues, matrix functions, or the solution of linear systems (which is are powerful tools to tackle many data science problems). A Hessenberg matrix $H_m\in\mathbb{R}^{m \times m}$ is defined by possessing the following sparsity structure:

$$
H_m = \begin{bmatrix}
\ast & \ast & \cdots & \ast & \ast\\
\ast & \ast & \cdots & \ast & \ast\\
 & \ast & \ddots & \vdots & \vdots\\
 & & \ddots & \ast & \ast\\
0 & & & \ast & \ast
\end{bmatrix}
$$

We now want to compute the QR decomposition of $H_m$. Of course, your functions from the previous tasks (as well as the second additional task) should work here, but we want to find an alternative way to tackle this problem.

**Task: Create a Hessenberg matrix $H_m$ and write a function to compute its QR decomposition WITHOUT using Householder reflections. The input should be $H_m$ and the output the two matrices $Q$ and $R$.**